In [1]:
#import packages
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

## Read Data

In [9]:
path = '/Users/leo/Desktop/DSGA-3001/Project/'

In [10]:
# Read in bureau
buro = pd.read_csv(path + 'data/bureau.csv')
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [11]:
buro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
SK_ID_CURR                int64
SK_ID_BUREAU              int64
CREDIT_ACTIVE             object
CREDIT_CURRENCY           object
DAYS_CREDIT               int64
CREDIT_DAY_OVERDUE        int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG        int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE               object
DAYS_CREDIT_UPDATE        int64
AMT_ANNUITY               float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [14]:
buro_bal = pd.read_csv(path+'data/bureau_balance.csv')
buro_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [15]:
buro_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
SK_ID_BUREAU      int64
MONTHS_BALANCE    int64
STATUS            object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


## Aggregation Functions

In [16]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values stats(count, mean, max, min and sum)
        groupby group_var in a dataframe.
    
    Args:
        df(dataframe):  
            the input dataframe
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
        agg (dataframe): 
            a dataframe with the stats aggregated for each group indexed by group_var
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    #select numeric columns + group_var
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = df[group_var]

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
                
    agg.columns = columns
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

In [17]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts of 
    each unique category for each observation groupby 'group_var'
    
    Args:
    df(dataframe):
        input dataframe
        
    group_var(string):
        The variable by which to group the dataframe. 
        
    df_name(string)
        Variable added to the front of column names to keep track of columns

    
    Returns:
    categorical(dataframe)
        A dataframe with counts and normalized counts of each unique category
        groupby the `group_var`.
        
    """
    
    # Select the categorical columns and do one-hot encoding
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # add the group_var as index
    categorical[group_var] = df[group_var]

    # calculate the sum and mean(count and normalized count) groupby group_var
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    #rename columns to df_name+column_name+stat_name
    for var in categorical.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stats in level 1
            for stat in ['count', 'count_norm']:
                column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    _, idx = np.unique(categorical, axis = 1, return_index=True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

In [18]:
def remove_duplicate_columns(df):
    _, idx = np.unique(df, axis = 1, return_index=True)
    df = df.iloc[:, idx]
    return df

## Preprocess

### 1. bureau.csv

In [19]:
#aggregate numerical features
buro_agg = agg_numeric(buro.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
buro_agg.head()

,bureau_DAYS_CREDIT_sum,bureau_DAYS_ENDDATE_FACT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,...,bureau_AMT_CREDIT_SUM_min,SK_ID_CURR,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max
0,-5145,-3302.0,-1572,-1329.0,-1328.0,-825.500000,-735.000000,-652,-544.0,-155,...,85500.0,100001,207623.571429,373239.0,378000.0,596686.5,1453365.000,NaN,NaN,NaN
1,-6992,-4185.0,-1437,-1072.0,-1185.0,-697.500000,-874.000000,-3999,-36.0,-1185,...,0.0,100002,108131.945625,245781.0,450000.0,245781.0,865055.565,0.0,1681.029,5043.645
2,-5603,-3292.0,-2586,-2434.0,-2131.0,-1097.333333,-1400.750000,-3264,-540.0,-2131,...,22248.0,100003,254350.125000,0.0,810000.0,0.0,1017400.500,0.0,0.000,0.000
3,-1734,-1065.0,-1326,-595.0,-683.0,-532.500000,-867.000000,-1064,-382.0,-682,...,94500.0,100004,94518.900000,0.0,94537.8,0.0,189037.800,0.0,0.000,0.000
4,-572,-123.0,-373,-128.0,-123.0,-123.000000,-190.666667,-163,-123.0,-121,...,29826.0,100005,219042.000000,543087.0,568800.0,568408.5,657126.000,0.0,0.000,0.000


In [20]:
#should be 12*5+1 columns
buro_agg.shape

(305811, 57)

In [21]:
#count categorical features
buro_counts = count_categorical(buro, group_var = 'SK_ID_CURR', df_name = 'bureau')
buro_counts.head()

,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count_norm,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_TYPE_Interbank credit_count_norm,bureau_CREDIT_TYPE_Interbank credit_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,...,bureau_CREDIT_TYPE_Credit card_count_norm,bureau_CREDIT_TYPE_Credit card_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_TYPE_Consumer credit_count_norm,bureau_CREDIT_CURRENCY_currency 1_count_norm,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_TYPE_Consumer credit_count,bureau_CREDIT_CURRENCY_currency 1_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.000000,0,0.428571,0.571429,1.000000,1.0,3,4,7,7
100002,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.500000,4,0.250000,0.750000,0.500000,1.0,2,6,4,8
100003,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.500000,2,0.250000,0.750000,0.500000,1.0,1,3,2,4
100004,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.000000,0,0.000000,1.000000,1.000000,1.0,0,2,2,2
100005,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.333333,1,0.666667,0.333333,0.666667,1.0,2,1,2,3


In [22]:
buro_counts.shape

(305811, 46)

### 2. bureau_balance.csv

#### groupby SK_ID_BUREAU

In [23]:
#aggregate numerical values
buro_bal_agg = agg_numeric(buro_bal, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
buro_bal_agg.head()

,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count,SK_ID_BUREAU
0,-4656,-96,-48.0,0,97,5001709
1,-3403,-82,-41.0,0,83,5001710
2,-6,-3,-1.5,0,4,5001711
3,-171,-18,-9.0,0,19,5001712
4,-231,-21,-10.5,0,22,5001713


In [24]:
# only one numerical col: MONTHS_BALANCE
# should be 1*5+1 columns
buro_bal_agg.shape

(817395, 6)

In [25]:
buro_bal_counts = count_categorical(buro_bal, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
buro_bal_counts.head()

,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_2_count_norm,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_3_count,bureau_balance_STATUS_2_count,bureau_balance_STATUS_5_count,bureau_balance_STATUS_1_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_0_count,bureau_balance_STATUS_X_count_norm,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_C_count
SK_ID_BUREAU,,,,,,,,,,,,,,,,
5001709,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,0.113402,0.886598,11,86
5001710,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.060241,5,0.361446,0.578313,30,48
5001711,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.750000,3,0.250000,0.000000,1,0
5001712,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.526316,10,0.000000,0.473684,0,9
5001713,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,1.000000,0.000000,22,0


In [26]:
buro_bal_counts.shape

(817395, 16)

#### then do aggregation again groupby SK_ID_CURR

In [27]:
# merge numeric stats and categorical count features generated above
bureau_by_loan = buro_bal_agg.merge(buro_bal_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge with bureau.csv to include the SK_ID_CURR
bureau_by_loan = buro[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')

bureau_by_loan.shape

(1716428, 23)

In [28]:
bureau_by_loan.columns

Index(['SK_ID_BUREAU', 'SK_ID_CURR', 'bureau_balance_MONTHS_BALANCE_sum',
       'bureau_balance_MONTHS_BALANCE_min',
       'bureau_balance_MONTHS_BALANCE_mean',
       'bureau_balance_MONTHS_BALANCE_max',
       'bureau_balance_MONTHS_BALANCE_count',
       'bureau_balance_STATUS_4_count_norm',
       'bureau_balance_STATUS_3_count_norm',
       'bureau_balance_STATUS_2_count_norm',
       'bureau_balance_STATUS_5_count_norm', 'bureau_balance_STATUS_4_count',
       'bureau_balance_STATUS_3_count', 'bureau_balance_STATUS_2_count',
       'bureau_balance_STATUS_5_count', 'bureau_balance_STATUS_1_count_norm',
       'bureau_balance_STATUS_1_count', 'bureau_balance_STATUS_0_count_norm',
       'bureau_balance_STATUS_0_count', 'bureau_balance_STATUS_X_count_norm',
       'bureau_balance_STATUS_C_count_norm', 'bureau_balance_STATUS_X_count',
       'bureau_balance_STATUS_C_count'],
      dtype='object')

In [29]:
# Aggregate the stats for each client
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
bureau_balance_by_client.shape

(305811, 86)

In [30]:
bureau_balance_by_client.columns

Index(['client_bureau_balance_MONTHS_BALANCE_sum_sum',
       'client_bureau_balance_MONTHS_BALANCE_sum_min',
       'client_bureau_balance_MONTHS_BALANCE_sum_mean',
       'client_bureau_balance_MONTHS_BALANCE_min_sum',
       'client_bureau_balance_MONTHS_BALANCE_mean_sum',
       'client_bureau_balance_MONTHS_BALANCE_min_min',
       'client_bureau_balance_MONTHS_BALANCE_mean_min',
       'client_bureau_balance_MONTHS_BALANCE_min_mean',
       'client_bureau_balance_MONTHS_BALANCE_mean_mean',
       'client_bureau_balance_MONTHS_BALANCE_sum_max',
       'client_bureau_balance_MONTHS_BALANCE_min_max',
       'client_bureau_balance_MONTHS_BALANCE_mean_max',
       'client_bureau_balance_MONTHS_BALANCE_max_sum',
       'client_bureau_balance_MONTHS_BALANCE_max_min',
       'client_bureau_balance_MONTHS_BALANCE_max_mean',
       'client_bureau_balance_STATUS_4_count_norm_sum',
       'client_bureau_balance_STATUS_5_count_norm_sum',
       'client_bureau_balance_STATUS_4_count_sum',
    

In [44]:
bureau_balance_by_client.isna().sum()

client_bureau_balance_MONTHS_BALANCE_sum_sum            0
client_bureau_balance_MONTHS_BALANCE_sum_min       171269
client_bureau_balance_MONTHS_BALANCE_sum_mean      171269
client_bureau_balance_MONTHS_BALANCE_min_sum            0
client_bureau_balance_MONTHS_BALANCE_mean_sum           0
client_bureau_balance_MONTHS_BALANCE_min_min       171269
client_bureau_balance_MONTHS_BALANCE_mean_min      171269
client_bureau_balance_MONTHS_BALANCE_min_mean      171269
client_bureau_balance_MONTHS_BALANCE_mean_mean     171269
client_bureau_balance_MONTHS_BALANCE_sum_max       171269
client_bureau_balance_MONTHS_BALANCE_min_max       171269
client_bureau_balance_MONTHS_BALANCE_mean_max      171269
client_bureau_balance_MONTHS_BALANCE_max_sum            0
client_bureau_balance_MONTHS_BALANCE_max_min       171269
client_bureau_balance_MONTHS_BALANCE_max_mean      171269
client_bureau_balance_STATUS_4_count_norm_sum           0
client_bureau_balance_STATUS_5_count_norm_sum           0
client_bureau_

# Merge with training data

In [31]:
train = pd.read_csv(path+'data/application_train.csv')
test = pd.read_csv(path+'data/application_test.csv')

In [32]:
original_features = list(train.columns)
print('Original Number of Features: ', len(original_features))

# Merge with the value counts of bureau
train = train.merge(buro_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
train = train.merge(buro_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the monthly information grouped by client
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

new_features = list(train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))

Original Number of Features:  122
Number of features using previous loans from other institutions data:  309


In [33]:
print('Original Number of Features: ', len(list(test.columns)))

# Merge with the value counts of bureau
test = test.merge(buro_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
test = test.merge(buro_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the value counts of bureau balance
test = test.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

print('Shape of Testing Data: ', test.shape)

Original Number of Features:  121
Shape of Testing Data:  (48744, 308)


# Align train and test set

In [34]:
train_labels = train['TARGET']

# Align the dataframes, this will remove the 'TARGET' column
train, test = train.align(test, join = 'inner', axis = 1)

train['TARGET'] = train_labels

print('Training Data Shape: ', train.shape)
print('Testing Data Shape: ', test.shape)

Training Data Shape:  (307511, 309)
Testing Data Shape:  (48744, 308)


# Save processed datafile

In [35]:
train.to_csv(path+'data/train_bureau.csv', index = False)
test.to_csv(path+'data/test_bureau.csv', index = False)

In [36]:
train = pd.read_csv(path+'data/train_bureau.csv')

In [40]:
train.columns.values

array(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE',
       'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE',
       'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG'